In [1]:
import pandas as pd
import numpy as np

import os

def load_hotel_reserve():
  customer_tb = pd.read_csv('../awesomebook-master/data/customer.csv')
  hotel_tb = pd.read_csv('../awesomebook-master/data/hotel.csv')
  reserve_tb = pd.read_csv('../awesomebook-master/data/reserve.csv')
  return customer_tb, hotel_tb, reserve_tb


def load_holiday_mst():
  holiday_tb = pd.read_csv('../awesomebook-master/data/holiday_mst.csv',
                           index_col=False)
  return holiday_tb


def load_production():
  production_tb = pd.read_csv('../awesomebook-master/data/production.csv')
  return production_tb


def load_production_missing_num():
  production_tb = pd.read_csv('../awesomebook-master/data/production_missing_num.csv')
  return production_tb


def load_production_missing_category():
  production_tb = pd.read_csv('../awesomebook-master/data/production_missing_category.csv')
  return production_tb


def load_monthly_index():
  monthly_index_tb = \
    pd.read_csv('../awesomebook-master/data/monthly_index.csv')
  return monthly_index_tb


def load_meros_txt():
  with open('../awesomebook-master/data/txt/meros.txt', 'r') as f:
    meros = f.read()
    f.close()
  return meros


In [2]:
customer_tb, hotel_tb, reserve_tb = load_hotel_reserve()

In [3]:
import pandas as pd

# 아래 부터 책에 게재
# 데이터형 확인
type(40000 / 3)

# 정수형으로 변환
int(40000 / 3)

# 부동 소수점형으로 변환
float(40000 / 3)

df = pd.DataFrame({'value': [40000 / 3]})

# 데이터형 확인
df.dtypes

# 정수형으로 변환
df['value'].astype('int8')
df['value'].astype('int16')
df['value'].astype('int32')
df['value'].astype('int64')

# 부동 소수점형으로 변환
df['value'].astype('float16')
df['value'].astype('float32')
df['value'].astype('float64')
df['value'].astype('float128')

# 아래와 같이 python의 데이터형을 지정할 수 있다
df['value'].astype(int)
df['value'].astype(float)


0    13333.333333
Name: value, dtype: float64

In [4]:
reserve_tb['total_price_log'] = \
  reserve_tb['total_price'].apply(lambda x: np.log10(x / 1000 + 1))

reserve_tb['total_price_log']

0       1.992111
1       1.334454
2       1.539076
3       2.290925
4       1.839478
          ...   
4025    1.230449
4026    1.631444
4027    1.879669
4028    2.733197
4029    1.654177
Name: total_price_log, Length: 4030, dtype: float64

In [5]:
customer_tb['age_rank'] = \
  (np.floor(customer_tb['age'] / 10) * 10).astype('category')

print(customer_tb['age_rank'])


0      40.0
1      30.0
2      40.0
3      40.0
4      30.0
       ... 
995    40.0
996    30.0
997    30.0
998    40.0
999    30.0
Name: age_rank, Length: 1000, dtype: category
Categories (7, float64): [20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0]


In [6]:
from sklearn.preprocessing import StandardScaler

# 소수점 이하를 다룰수 있게 하기 위해 float형으로 변환
reserve_tb['people_num'] = reserve_tb['people_num'].astype(float)

# 정규화를 실행할 오브젝트를 생성
ss = StandardScaler()

# fit_transform 함수는 fit 함수(정규화 하기위한 준비 계산)와 
# transform 함수 (준비된 정보에서 정규화 변환 처리를 실행)를 모두 실행
result = ss.fit_transform(reserve_tb[['people_num', 'total_price']])

reserve_tb['people_num_normalized'] = [x[0] for x in result]
reserve_tb['total_price_normalized'] = [x[1] for x in result]

reserve_tb['people_num_normalized']

0       1.300709
1      -0.483753
2      -0.483753
3       1.300709
4       0.408478
          ...   
4025   -0.483753
4026   -0.483753
4027   -0.483753
4028    1.300709
4029   -1.375984
Name: people_num_normalized, Length: 4030, dtype: float64

In [7]:
reserve_tb = reserve_tb[
  (abs(reserve_tb['total_price'] - np.mean(reserve_tb['total_price'])) /
   np.std(reserve_tb['total_price']) <= 3)
].reset_index()


In [ ]:
production_tb = load_production()

# PCA 블러오기
from sklearn.decomposition import PCA

# n_components에 주성분 분석으로 변환 후의 차원수를 지정
pca = PCA(n_components=2)

# 주성분 분석 실행
# pca에 주성분 분석 변환 파라미터가 저장되어, 반환 값에 주성분 분석 후의  값이 반환된다.
pca_values = pca.fit_transform(production_tb[['length', 'thickness']])

# 누적 기여율과 기여율 확인
print('누적 기여울: {0}'.format(sum(pca.explained_variance_ratio_)))
print('각 차원의 기여율: {0}'.format(pca.explained_variance_ratio_))

# predict 함수를 이용하여 동일한 차원 압축 처리를 실행
pca_newvalues = pca.transform(production_tb[['length', 'thickness']])

pca_newvalues

누적 기여울: 0.9999999999999999
각 차원의 기여율: [0.97897794 0.02102206]


array([[  76.96838157,   13.38906936],
       [-112.11469337,    8.24884796],
       [ -76.1994339 ,  -11.19027127],
       ...,
       [  31.12100559,  -15.48152593],
       [-117.87675543,   -2.4361334 ],
       [   4.80243541,   15.32174872]], shape=(1000, 2))

In [17]:
production_miss_num = load_production_missing_num()

# replace함수로 None을 nan으로 변환
# （None을 지정할 때는 문자열로 지정해야한다.)
production_miss_num.replace('None', np.nan, inplace=True)

# dropna 함수로 thickness에 nan이 포함된 레코드를 제거
production_miss_num.dropna(subset=['thickness'], inplace=True)

In [18]:
# replace 함수로 None을 nan로 변환
production_miss_num.replace('None', np.nan, inplace=True)

# fillna 함수로 thickness의 결손값을 1로 보완
production_miss_num['thickness'].fillna(1, inplace=True)


/tmp/ipykernel_30583/2603283322.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  production_miss_num['thickness'].fillna(1, inplace=True)


In [19]:
# replace 함수로 None을 nan으로 변환
production_miss_num.replace('None', np.nan, inplace=True)

# thickness를 수치형으로 변환(None이 섞여 있기 때문에 수치형이 아닌 상태이다)
production_miss_num['thickness'] = \
  production_miss_num['thickness'].astype('float64')

# thickness의 평균값을 계산
thickness_mean = production_miss_num['thickness'].mean()

# thickness의 결손 값을 thickness의 평균값으로 보완
production_miss_num['thickness'].fillna(thickness_mean, inplace=True)



/tmp/ipykernel_30583/3312858142.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  production_miss_num['thickness'].fillna(thickness_mean, inplace=True)
